<a href="https://colab.research.google.com/github/adpenev/stf/blob/master/civic_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install pipenv
!pipenv install civic-scraper
!python -m pip install civic-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Installing civic-scraper...
Resolving civic-scraper...
Installing...
Adding civic-scraper to Pipfile's [packages] ...
✔ Installation Succeeded
⠸ Installing civic-scraper...
Installing dependencies from Pipfile.lock (396dbd)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!ls /root/.civic-scraper/metadata/

ls: cannot access '/root/.civic-scraper/metadata/': No such file or directory


In [ ]:
!wc -l /root/.civic-scraper/metadata/civic_scraper_assets_meta_20230405T1848z.csv

wc: /root/.civic-scraper/metadata/civic_scraper_assets_meta_20230405T1848z.csv: No such file or directory


In [ ]:
!cat /root/.civic-scraper/metadata/civic_scraper_assets_meta_20230405T1848z.csv

cat: /root/.civic-scraper/metadata/civic_scraper_assets_meta_20230405T1848z.csv: No such file or directory


In [ ]:
from civic_scraper.platforms import CivicPlusSite

url = "https://www.tuolumnecounty.ca.gov/AgendaCenter"
site = CivicPlusSite(url)
site.scrape(download=True)

[Asset(https://www.tuolumnecounty.ca.gov/AgendaCenter/ViewFile/Agenda/_04062023-2109),
 Asset(https://www.tuolumnecounty.ca.gov/AgendaCenter/ViewFile/Agenda/_04062023-2109),
 Asset(https://www.tuolumnecounty.ca.gov/AgendaCenter/ViewFile/Agenda/_04062023-2113),
 Asset(https://www.tuolumnecounty.ca.gov/AgendaCenter/ViewFile/Agenda/_04062023-2113)]

In [ ]:
from civic_scraper.platforms import CivicPlusSite

url = "https://www.tuolumnecounty.ca.gov/AgendaCenter"
site = CivicPlusSite(url)
assets_metadata = site.scrape()
assets_metadata
assets_metadata.to_csv('/tmp/civic-scraper/metadata')

'/tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv'

In [ ]:
df = pd.read_csv('/tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2000z.csv')

df.head(2)

,place,state_or_province,meeting_date,meeting_time,committee_name,meeting_id,asset_name,asset_type,url,scraped_by,content_type,content_length
0,no_data,no_data,2023-04-06 00:00:00,NaN,Airport Land Use Commission,civicplus_www_04062023-2109,ALUC Cancellation Notice 4-6-23,agenda,https://www.tuolumnecounty.ca.gov/AgendaCenter...,civic-scraper_0.1.0,application/pdf,57683
1,no_data,no_data,2023-04-06 00:00:00,NaN,Airport Land Use Commission,civicplus_www_04062023-2109,ALUC Cancellation Notice 4-6-23,agenda,https://www.tuolumnecounty.ca.gov/AgendaCenter...,civic-scraper_0.1.0,application/pdf,57683


In [ ]:
civic_plus_sites_df = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1hwdLfNY1YKLA72yDHT3M-Gt1F_4AG1ueebT_lST30WU' +
                   '/export?gid=0&format=csv',
                  )
civic_plus_sites_df.shape

(1532, 10)

In [ ]:
# url = civic_plus_sites_df['url'].values[0]
# site = CivicPlusSite(url)
# assets_metadata = site.scrape(download=True)
# path = assets_metadata.to_csv('/tmp/civic-scraper/metadata')
# cp_df = pd.read_csv(path)
# has_data = []
# error_urls = []

# for i in range(0, civic_plus_sites_df.shape[0]):
#   if str(civic_plus_sites_df['data_status'].values[i]).strip() == 'error':
#     continue

#   url = civic_plus_sites_df['url'].values[i]
#   print(url)
#   site = CivicPlusSite(url)
#   # print('data_status: ' + str(civic_plus_sites_df['data_status'].values[i]).strip())
#   try: 
#     assets_metadata = site.scrape(download=True)
#     path = assets_metadata.to_csv('/tmp/civic-scraper/metadata')
#     print(str(i) + '. ' + path)
#     site_df = pd.read_csv(path)

#     if site_df.shape[0] > 0:
#       has_data.append(url)
#     print('site_data: ' + str(site_df.shape[0]))
#     cp_df = pd.concat([cp_df, site_df], ignore_index=True)
#     print(cp_df.shape[0])
#   except:
#     print('Error url:', url)
#     error_urls.append(url)

# print("URLs with data: ", has_data)
# print("URLs with errors: ", error_urls)



In [ ]:
has_data = []
error_urls = []
def scrape_url(row):
  # if str(row['data_status']).strip() == 'error':
  #   return
  
  url = row['url']
  print(url)
  site = CivicPlusSite(url)
  # print('data_status: ' + str(civic_plus_sites_df['data_status'].values[i]).strip())
  try: 
    assets_metadata = site.scrape(download=True)
    path = assets_metadata.to_csv('/tmp/civic-scraper/metadata')
    print(str(row.name) + '. ' + path)
    site_df = pd.read_csv(path)

    if site_df.shape[0] > 0:
      row['data_status']='has_data'
      has_data.append(url)
    elif site_df.shape[0] == 0:
      row['data_status']='no_data'

    row['files_count'] = site_df.shape[0]  
    print('site_data: ' + str(site_df.shape[0]))
    # cp_df = pd.concat([cp_df, site_df], ignore_index=True)
    # print(cp_df.shape[0])
  except:
    print('Error url:', url)
    row['data_status']='error'
    error_urls.append(url)
    row['files_count'] = 0
  
  return row

# civic_plus_sites_df1 = civic_plus_sites_df.head(30)
scraped_urls_df = civic_plus_sites_df.apply(scrape_url, axis=1)

print("URLs with data: ", has_data)
print("URLs with errors: ", error_urls)

https://banff.ca/AgendaCenter
0. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
https://www.beaumont.ab.ca/AgendaCenter
1. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
https://md.bonnyville.ab.ca/AgendaCenter
2. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
https://www.brooks.ca/AgendaCenter
3. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
https://www.chestermere.ca/AgendaCenter
4. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
https://www.cochrane.ca/AgendaCenter
5. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
http://ab-highlevel.civicplus.com/AgendaCenter
6. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2236z.csv
site_data: 0
http://ab-hintoncommunity.civicplus.com/AgendaCenter
7. /tmp/civic-scraper/metadata/civic_scraper_assets_me

441. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2249z.csv
site_data: 0
https://www.peoriaelections.org/AgendaCenter
Error url: https://www.peoriaelections.org/AgendaCenter
https://www.pcchd.org/AgendaCenter
443. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2249z.csv
site_data: 0
https://www.villageofpingreegrove.org/AgendaCenter
444. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2249z.csv
site_data: 0
https://www.cityofplanoil.com/AgendaCenter
445. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2250z.csv
site_data: 0
https://www.pontiac.org/AgendaCenter
446. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2250z.csv
site_data: 0
http://il-pontoonbeach.civicplus.com/AgendaCenter
447. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2250z.csv
site_data: 0
https://www.prospect-heights.il.us/AgendaCenter
448. /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2250z.csv
site_data: 

In [ ]:
scraped_urls_df.shape

(1532, 11)

In [ ]:
scraped_urls_df.head(5)

,name,state,country,gov_level,county_fips,state_fips,site_type,url,scrape,data_status,files_count
0,Banff,AB,CANADA,municipality,NaN,NaN,civicplus,https://banff.ca/AgendaCenter,NaN,no_data,0
1,Beaumont,AB,CANADA,municipality,NaN,NaN,civicplus,https://www.beaumont.ab.ca/AgendaCenter,NaN,no_data,0
2,Bonnyville,AB,CANADA,municipality,NaN,NaN,civicplus,https://md.bonnyville.ab.ca/AgendaCenter,NaN,no_data,0
3,Brooks,AB,CANADA,municipality,NaN,NaN,civicplus,https://www.brooks.ca/AgendaCenter,NaN,no_data,0
4,Chestermere,AB,CANADA,municipality,NaN,NaN,civicplus,https://www.chestermere.ca/AgendaCenter,NaN,no_data,0


In [ ]:
scraped_urls_df['files_count'].sum()

992

In [ ]:
# scraped_urls_df = scraped_urls_df.drop([109])
scraped_urls_df.to_excel("civic_plus_sites.xlsx", index=False)

In [ ]:
# /tmp/civic-scraper/metadata/civic_scraper_assets_meta_20230406T2105z.csv
!ls /tmp/civic-scraper/metadata